# Overview

An overview of the data collected during our monitoring period. 

---

In this notebook we aim to investigate the distribution of tweets, retweets, replies, hashtags and links within our dataset. By doing this we hope to gain some understanding of the data we have collected. It will also allow us to determine how active the two distinct groups of users, journalists and news organisations, are on social media.

In [ ]:
import collections
import pymongo
import json
import matplotlib.pyplot as plt
import numpy as np
import sys

# add penemue to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from utils import twiterate

%matplotlib inline

We begin by creating a connection to the database in which we stored the results of our data collection.

In [ ]:
db = MongoClient("mongodb://localhost:27017")["media-monitor"]